In [1]:
##########################################################################
##### Template code and instructions to scrape Rate My Professor Comments#
##########################################################################
### Step 1: install packages - 
##### packages are necessary to install and load, given that they have the built in functions necessary to run complex tasks. 
## They effectively act as one of the most crucial time saving activities that would otherwise lead to overly long and 
## duplicative scripts. 

## install pkgs 
import sys
!{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install requests #; this code here can be used to install packages on anaconda/jupyter notebook 
### I believe the below should be installed by default 
import requests # web scraping 
from bs4 import BeautifulSoup # for web scraping 
import itertools # for efficient operation of loops 
import pandas as pd # necessary for reading in, creating, and manipulating data frames 
import csv ## for importing/exporting csvs 

In [8]:
### Step 2: Read in the csv with the data of interest 
## note: This will be the section that you update manually the most. The rest should be automated 

faculty_csv = pd.read_csv('temp_unc_faculty_chr.csv')
urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv

,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,notes
0,UNIVERSITY OF NORTH CAROLINA,DALE,WHITTINGTON,ENVIRONMENTAL SCIENCES & ENGINEERING,1,0,https://www.ratemyprofessors.com/professor/100...,NaN
1,UNIVERSITY OF NORTH CAROLINA,COURTNEY,WOODS,ENVIRONMENTAL SCIENCES & ENGINEERING,0,1,https://www.ratemyprofessors.com/professor/225...,NaN
2,UNIVERSITY OF NORTH CAROLINA,KARI,NORTH,EPIDEMIOLOGY,0,0,https://www.ratemyprofessors.com/professor/212...,NaN
3,UNIVERSITY OF NORTH CAROLINA,SUZANNE,MAMAN,HEALTH BEHAVIOR,0,0,https://www.ratemyprofessors.com/professor/286...,Ratings are for Health Science
4,UNIVERSITY OF NORTH CAROLINA,BENJAMIN,MEIER,HEALTH POLICY & MANAGEMENT,1,0,https://www.ratemyprofessors.com/professor/143...,Ratings are for Public Policy
5,UNIVERSITY OF NORTH CAROLINA,AMANDA,THOMPSON,NUTRITION,0,0,https://www.ratemyprofessors.com/professor/213...,Ratings are for Anthropology
6,UNIVERSITY OF NORTH CAROLINA,ASHLEY,ANDERSON,POLITICAL SCIENCE,0,1,https://www.ratemyprofessors.com/professor/243...,NaN
7,UNIVERSITY OF NORTH CAROLINA,CAMERON,BALLARD-ROSA,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/231...,NaN
8,UNIVERSITY OF NORTH CAROLINA,NAVIN,BAPAT,POLITICAL SCIENCE,1,1,https://www.ratemyprofessors.com/professor/106...,NaN
9,UNIVERSITY OF NORTH CAROLINA,ANNA,BASSI,POLITICAL SCIENCE,0,0,https://www.ratemyprofessors.com/professor/164...,NaN


In [9]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "class_code": prof_class[i].get_text(),
            "college": college[j], #jth lvl, not ith level, variable 
            "prof_firstname": prof_firstname[j],            
            "prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ### now save the file 
    save_name = "scraped_data/rmp_data" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)





